
# Анализ дорожно-транспортных проишествий по Кировской области 

В этой работе проводится анализ данных «Карты ДТП» — некоммерческого проекта, посвящённого проблеме дорожно-транспортных происшествий в России.

Заказчика интересует, с чем может быть связана аварийность на дорогах.

- Автор: Юрий Кузнецов

- Дата: 05.03.2025


## Что нужно сделать
Выполнить предобработку для удобства работы с данными: выбрать оптимальные названия столбцов и типы данных. Вам также понадобится дать ответы на вопросы заказчика.

### Вопросы заказчика

1. Как распределено количество участников ДТП и почему? Встречаются ли аномальные значения или выбросы? Если да, то с чем они могут быть связаны? Для числа участников найти наиболее типичное значение. Проверить распределения и других столбцов.

2. Между какими столбцами в датасете `Kirovskaya_oblast.csv` высокая корреляция. Проверить своё предположение.  

3. Как связаны категории аварий и погодные условия?

4. Построить процентную разбивку аварий по видам освещённости с усетом пола участника и сделать расчёты для мужчин и женщин отдельно.

5. Исследовать, чем отличаются аварии без пострадавших от тех, в которых был один пострадавший или более.

6. Сделать общий вывод о связи аварийности с другими факторами, а также рекомендации.

## Описание данных

Датасет `Kirovskaya_oblast.csv` содержит информацию о ДТП в Кировской области:

* `geometry.coordinates` — координаты ДТП;

* `id` — идентификатор ДТП;

* `properties.tags` — тег происшествия;

* `properties.light` — освещённость;

* `properties.point.lat` — широта;

* `properties.point.long` — долгота;

* `properties.nearby` — ближайшие объекты;

* `properties.region` — регион;

* `properties.scheme` — схема ДТП;

* `properties.address` — ближайший адрес;

* `properties.weather` — погода;

* `properties.category` — категория ДТП;

* `properties.datetime` — дата и время ДТП;

* `properties.injured_count` — число пострадавших;

* `properties.parent_region` — область;

* `properties.road_conditions` — состояние покрытия;

* `properties.participants_count` — число участников;

* `properties.participant_categories` — категории участников.

`Kirovskaya_oblast_participiants.csv` хранит сведения об участниках:

* `role` — роль;

* `gender` — пол;

* `violations` — какие правила дорожного движения были нарушены конкретным участником;

* `health_status` — состояние здоровья после ДТП;

* `years_of_driving_experience` — число лет опыта;

* `id` — идентификатор ДТП.

### Подключение библиотек и загрузка данных

In [ ]:
 !pip install phik 


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from phik import phik_matrix

### Загрузка датасета с информацией о ДТП по Кировской области

In [ ]:
df_dtp_info_k = pd.read_csv('https://code.s3.yandex.net/datasets/Kirovskaya_oblast.csv')

In [ ]:
df_dtp_info_k.head()

#### Оптимизация назаваний столбцов 


In [ ]:
df_dtp_info_k.columns = [c.replace('properties.', '') for c in df_dtp_info_k.columns]
df_dtp_info_k.columns = [c.replace('.', '_') for c in df_dtp_info_k.columns]

In [ ]:
df_dtp_info_k.info()

df_dtp_info_k содержит 14517 строк в 18 столбцах

#### Проверим пропуски в датасете df_dtp_info_k с информацией о ДТП

In [ ]:
missing = (pd.DataFrame({'Кол-во пропусков': df_dtp_info_k.isnull().sum(), 'Процент пропусков': round(df_dtp_info_k.isnull().mean().round(5), 3)*100}).sort_values(by='Кол-во пропусков', ascending=False)
           .style.background_gradient(cmap='coolwarm'))
missing

In [ ]:
df_dtp_info_k[['scheme', 'address']] = df_dtp_info_k[['scheme', 'address']].fillna('нет данных')

Пропуски содержатся в столбцах scheme (7,8%), address (4,6%), заполним их значением 'нет данных'. point_lat (0,2%), point_long (0,2%), которые не понадобятся нам для данного исследования. Удалять или заполнять их нецелесообразно 

#### Обработка дубликатов в датасете df_dtp_info_k

In [ ]:
df_dtp_info_k.info()

In [ ]:
# Приведем данные текстовых столбцах df_dtp_info
for column in ['tags', 'light', 'region', 'address', 'weather','category', 'parent_region', 'road_conditions', 'participant_categories']:
    df_dtp_info_k[column]=df_dtp_info_k[column].apply(lambda x:x.lower().strip())

Проверим наличие явных дубликатов

In [ ]:
df_dtp_info_k.duplicated().sum()

Явных дубликатов нет

In [ ]:
df_dtp_info_k['id'].duplicated().sum()

Все id уникальны, то есть неявных дубликатов тоже нет

#### Проверим и оптимизируем типы данных датасета df_dtp_info_k

In [ ]:
df_dtp_info_k.info()

id следует перевести в тип object

datetime привести к datetime

injured_count и participants_count следует понизить разрядность

In [ ]:
df_dtp_info_k['id']=df_dtp_info_k['id'].astype('object')

In [ ]:
df_dtp_info_k['datetime']=df_dtp_info_k['datetime'].astype('datetime64[ns]')

In [ ]:
df_dtp_info_k['injured_count']=pd.to_numeric(df_dtp_info_k['injured_count'], downcast='integer')
df_dtp_info_k['participants_count']=pd.to_numeric(df_dtp_info_k['participants_count'], downcast='integer')

In [ ]:
#for column in ['tags', 'light', 'region','weather','category', 'parent_region', 'road_conditions', 'participant_categories']:
#    df_dtp_info_k[column] = df_dtp_info_k[column].astype('category')

In [ ]:
df_dtp_info_k.info()

### Загрузка датасета с информацией об участниках ДТП по Кировской области


In [ ]:
df_dtp_participants_k = pd.read_csv('https://code.s3.yandex.net/datasets/Kirovskaya_oblast_participiants.csv')

In [ ]:
df_dtp_participants_k.head()

In [ ]:
df_dtp_participants_k.info()

df_dtp_participants_k 31235 строк в 6 столбцах содержит информацию об участниках в Кировской области

#### Проверим пропуски в датасете df_dtp_participants_k с информацией об участниках ДТП

In [ ]:
missing_p = (pd.DataFrame({'Кол-во пропусков': df_dtp_participants_k.isnull().sum(), 'Процент пропусков': round(df_dtp_participants_k.isnull().mean().round(5), 3)*100}).sort_values(by='Кол-во пропусков', ascending=False)
           .style.background_gradient(cmap='coolwarm'))
missing_p

39,4 % пропусков в поле years_of_driving_experience, лет вождения, что модет говорить о водителях с отсутствующим стажем вождения. Предполагаем заменить на значение 0.
Пол водителя не указан в 2.2 % данных, что является случайным. Предполагается заменить на индикатор "нет данных"
Состояние здоровья не указано в 0.2 %. Что может означать что здоровье осталось в норме после дтп. Однако это лишь предположение. Предполагается заменить на моду


In [ ]:
#df_dtp_participants['years_of_driving_experience']=df_dtp_participants['years_of_driving_experience'].fillna(0)
#df_dtp_participants_k['gender']=df_dtp_participants_k['gender'].fillna('нет данных')
df_dtp_participants_k['health_status']=df_dtp_participants_k['health_status'].fillna('Не пострадал')

In [ ]:
df_dtp_participants_k['health_status'].mode()

In [ ]:
missing_p = (pd.DataFrame({'Кол-во пропусков': df_dtp_participants_k.isnull().sum(), 'Процент пропусков': round(df_dtp_participants_k.isnull().mean().round(5), 3)*100}).sort_values(by='Кол-во пропусков', ascending=False)
           .style.background_gradient(cmap='coolwarm'))
missing_p

Пропуски в столбце years_of_driving_experience оставим как есть. Они по видимому неслучайны, но об их наличии мы ничего не можем сказать на основе имеющихся данных

#### Проверим дубликаты в датасете df_dtp_participants

In [ ]:
# Приведем данные текстовых столбцах df_dtp_info
for column in ['role', 'violations', 'health_status']:
    df_dtp_participants_k[column]=df_dtp_participants_k[column].apply(lambda x:x.lower().strip())

In [ ]:
df_dtp_participants_k.duplicated().sum()

In [ ]:
round(df_dtp_participants_k.duplicated().sum()/df_dtp_participants_k.shape[0], 3)*100

Обнаружено наличие явных дубликатов 31,1%. Удалим их

In [ ]:
df_dtp_participants_k = df_dtp_participants_k.drop_duplicates()


In [ ]:
df_dtp_participants_k.info()

Поищем неявные

In [ ]:
df_dtp_participants_k[['role', 'gender', 'violations', 'health_status', 'id']].duplicated().sum()

In [ ]:
df_dtp_participants_k['id'].duplicated().sum()

В данном датасете значение id может повторяться, так как в одном ДТП может быть более одного участника

Неявных дубликатов не обнаружено

#### Проверка и оптимизация типов данных df_dtp_participants_k

In [ ]:
df_dtp_participants_k.info()

тип данных years_of_driving_experience следует привести к целочисленному типу

In [ ]:
df_dtp_participants_k['years_of_driving_experience']=df_dtp_participants_k['years_of_driving_experience'].astype('Int64')
df_dtp_participants_k['years_of_driving_experience']=pd.to_numeric(df_dtp_participants_k['years_of_driving_experience'], downcast='integer')

Значения id к типу object

In [ ]:
df_dtp_participants_k['id']=df_dtp_participants_k['id'].astype('object')

In [ ]:
df_dtp_participants_k.info()

### Объединим датасеты df_dtp_info_k и df_dtp_participants_k

In [ ]:
df=df_dtp_info_k.merge(df_dtp_participants_k, on='id', how='outer')

In [ ]:
df.head()

In [ ]:
df.info()

Общий датасет df имеет 22685 строк и 23 столбца, содержит информацию о дорожно-транспортных происшествиях и участниках. Все пропуски и дубликаты обработаны, ти


## Исследование данных

### 1. Как распределено количество участников ДТП и почему? Встречаются ли аномальные значения или выбросы? Если да, то с чем они могут быть связаны? Для числа участников найти наиболее типичное значение. Проверить распределения и других столбцов.

#### Исследуем участников ДТП

In [ ]:
df['participants_count'].describe()

Среднее значение числа участников отличается в большую сторону от медианы, следует ожидать правосимметричного распределения

Посмотрим распределение на диаграмме размаха

In [ ]:
plt.figure(figsize=(18, 6))
sns.boxplot(data=df, x='participants_count')

plt.title('Диаграмма размаха числа участников ДТП')
plt.grid()
plt.show()

Действительно имеются выбросы значений, которые могут интерпретироваться редкими случаями ДТП
Значение 30 участников аномально высокое, возможно ошибка ввода данных

Посмотрим информацию по данному ДТП

In [ ]:
df[df['participants_count']==30]

Число пострадавших и число участников ДТП одинаковое, не похоже на ошибку ввода данных. Поиск в интернете показал, что такое ДТП действительно случилось с пассажирским автобусом в Кировской области по этому адресу.

Не будем учитывать это значение чтобы определить наиболее типичное значение числа участников ДТП

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=df[df['participants_count']< 30], x='participants_count', bins=30)

plt.title('Распределение участников ДТП')
plt.ylabel('Количество ДТП')
plt.xlabel('Число участноков')
plt.grid()
plt.show()

- Наиболее типичное число участников ДТП 2.

In [ ]:
df_participants = pd.DataFrame({'Число ДТП': df.groupby('participants_count')['id'].nunique(),
                             'Процент': round(df.groupby('participants_count')['id'].nunique()/df.groupby('participants_count')['id'].nunique().sum(), 4)*100
                            }).head(15)

df_participants_sorted = df_participants.sort_values(by='Число ДТП', ascending=False).reset_index()
df_participants_sorted = df_participants_sorted.rename(columns={'participants_count':'Число участников'})
df_participants_sorted['№'] = range(1, len(df_participants_sorted) + 1)
df_participants_sorted = df_participants_sorted.set_index('№')
df_participants_sorted=df_participants_sorted.style.background_gradient(cmap='coolwarm')
df_participants_sorted

- 59,3 % случаев ДТП происходит с 2 участниками, это вполне логично, учитывая, что как правило в ДТП учавствуют 2 стороны: водитель-водитель или водитель-пешеход.
- 20,2 % с 3 участниками, по-видимому с участием еще и пассажира
- 10,6 с одним в случаях ДТП с одним ТС, например при сьезде с дороги

#### Проанализируем распределение пострадавших в ДТП

In [ ]:
df['injured_count'].describe()

Среднее значение также отличается от медианы в большую сторону, что говорит о высоких значения выбросов

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=df[df['injured_count']< 30], x='injured_count', bins=30)

plt.title('Распределение пострадавших в ДТП')
plt.ylabel('Количество ДТП')
plt.xlabel('Число пострадавших')
plt.grid()
plt.show()

In [ ]:
df_injured = pd.DataFrame({'Число ДТП': df.groupby('injured_count')['id'].nunique(),
                             'Процент': round(df.groupby('injured_count')['id'].nunique()/df.groupby('injured_count')['id'].nunique().sum(), 4)*100
                            }).head(15)

df_injured_sorted = df_injured.sort_values(by='Число ДТП', ascending=False).reset_index()
df_injured_sorted = df_injured_sorted.rename(columns={'injured_count':'Число пострадавших'})
df_injured_sorted['№'] = range(1, len(df_injured_sorted) + 1)
df_injured_sorted = df_injured_sorted.set_index('№')
df_injured_sorted=df_injured_sorted.style.background_gradient(cmap='coolwarm')
df_injured_sorted

- В 79,6 % случаев ДТП имеется один пострадавший, двое в 13,5 % происшествий, трое в 4,2 %

### 2. Предположение и проверка корреляции между столбцами

Для этого применим рассчет коэффициента корреляции phi k

In [ ]:
correlation_matrix = df[['light', 'weather', 'road_conditions', 'gender', 'health_status',
                         'years_of_driving_experience', 'role', 'injured_count']].phik_matrix()
plt.figure(figsize=(2, 6))

data_heatmap = correlation_matrix.loc[correlation_matrix.index != 'injured_count'][['injured_count']].sort_values(by='injured_count', ascending=False)
sns.heatmap(data_heatmap,
            annot=True, 
            fmt='.2f', 
            cmap='coolwarm', 
            linewidths=0.5, 
            cbar=False 
           )
plt.title('Тепловая карта коэффициента phi_k \n для данных rating')
plt.show()

Очевидна сильная корреляция между пострадавшими и дорожным покрытием. Следует глубже исследовать эту взаимосвязь

Развернем списки в столбце road_conditions в отдельные строки для датафрейма df_exploded

In [ ]:
import ast

In [ ]:

df_exploded = df.copy()
df_exploded["road_conditions"] = df_exploded["road_conditions"].apply(ast.literal_eval)
df_exploded = df_exploded.explode("road_conditions")

Выведем число ДТП в зависимости от состояния дорожного покрытия

In [ ]:
df_exp = df_exploded.groupby('road_conditions')['id'].nunique().sort_values(ascending = False).reset_index().head(10)
df_exp['mean']=df_exp['id']/df_exp['id'].sum()
df_exp.style.background_gradient(cmap='coolwarm')

Выясним как суммарное число пострадавших зависит от состояния дорожного покрытия зависит 

In [ ]:
#df_exploded=df_exploded[df_exploded['injured_count']>1]
df_injured = df_exploded.groupby(['road_conditions', 'injured_count'])['id'].nunique().reset_index().sort_values(by=[ 'id'], ascending=False)
df_injured_sum = df_injured.groupby('road_conditions')['injured_count'].sum().sort_values( ascending=False)
df_injured_sum.reset_index().head(10).style.background_gradient(cmap='coolwarm')

Отобразим на линейчатой диаграмме

In [ ]:

df_injured_sum=df_injured_sum.reset_index().head(15)

# Построение диаграммы
plt.figure(figsize=(12, 10))
ax=sns.barplot(x= 'injured_count', y='road_conditions', data=df_injured_sum, color='red')
plt.title('Антитоп-15 состояний дорожных покрытий по числу  пострадавших', fontsize=20)
plt.xlabel('Суммарное число пострадавших', fontsize=15)
plt.ylabel(None)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.grid()
plt.show()

- Чаще всего на число пострадавших в ДТП влияет отсутствие, плохая различимость горизонтальной разметки проезжей части

### 3. Связь категории аварий с погодными условиями

Выведем количество ДТП сгруппированому по категоии аварий и погодным условиям

In [ ]:
df_group_cat_wet = df.groupby(['category', 'weather'])['id'].nunique().sort_values(ascending=False).head(20).reset_index()

df_group_cat_wet


Выведем число случаев ДТП по категория ДТП

In [ ]:
category= pd.DataFrame(df['category'].value_counts()).reset_index()
category

Число ДТП по погодным условиям

In [ ]:
weather= pd.DataFrame(df['weather'].value_counts()).reset_index().style.background_gradient(cmap='coolwarm')
weather

Построим тепловую карту числа ДТП по категориям ДТП от погодных условий

In [ ]:
# Создаем таблицу сопряженности
cross_table = pd.crosstab(df['category'], df['weather'])

# Тепловая карта
plt.figure(figsize=(10, 10))
sns.heatmap(
    cross_table, 
    annot=True,   
    cmap='coolwarm', 
    fmt='d',
)
plt.title('Число ДТП по категориям от погодных условий ')
plt.xlabel('Погода')
plt.ylabel('Категоря ДТП')
plt.show()

- Самые частые категории ДТП `столкновение`, `наезд на пешехода` и `съезд с дороги`
- Самые частые погодные условия `пасмурно`, `ясно`, `снегопад`, `дождь`

### 4. Процентная разбивка аварий по видам освещённости с учетом пола участника и сделаем расчёты для мужчин и женщин отдельно.

Посчитаем процент ДТП в зависимости от освещения с участием мужчин от общего числа мужчин

In [ ]:
mans_light = round(df[df['gender']=='Мужской'].groupby('light')['gender'].count()/
      ((df['gender']=='Мужской').sum()), 3)*100
mans_light

Посчитаем процент ДТП в зависимости от освещения с участием женщин от общего числа женщин

In [ ]:
womans_light = round(df[df['gender']=='Женский'].groupby('light')['gender'].count()/
      ((df['gender']=='Женский').sum()), 3)*100
womans_light

In [ ]:
mans_womans_light = pd.DataFrame({'Мужчины': mans_light,
                                  'Женщины': womans_light
    
})
mans_womans_light

In [ ]:
df['light'].value_counts()

In [ ]:
mans_womans_light.plot(kind='barh',
               title='Процент ДТП от освещенности по полу участника ',
               legend=True,
               xlabel='Освещенность',
               ylabel='Процент, %',
               rot=0,
               edgecolor = 'black',
               figsize=(7, 7))
plt.grid()
plt.show()

- Для мужчин и женщин в целом характерны приблизительно равные доли участия в ДТП.
- Участия женщин больше, чем мужчин в светлое время суток (64.4% и 63.8% соответственно) и темное с включенным освещением (22.4% и 21.8% соответственно)
- Участия мужчин больше чем женщин в сумерках (3.1% и 2.7% соответственно), в темное время суток, освещение не включено (1.5% и 1.3% соответственно) и в в темное время суток, освещение отсутствует (9.8% и 9.2% соответственно)
- В затемненных условиях участие в ДТП мужчин больше, а в условиях с освещением больше участие женщин

### 5. Исследование отличия аварий без пострадавших от тех, в которых был один пострадавший или более.

In [ ]:
df.head()

Создадим столбец с бинарными значениями, в котором 0 - ДТП без пострадавших, 1 - с пострадавшими

In [ ]:
df['binary'] = np.where(df['health_status'] == 'не пострадал', 0, 1)

In [ ]:
df['binary'].value_counts()

Для того чтобы определить условия, которые влияют на значения в бинарном столбце построим матрицу корреляции других признаков с этим столбцом 

In [ ]:
# Вычисляем корреляционную матрицу с использованием phi_k
correlation_matrix = df[['light', 'tags', 'region', 'scheme', 'parent_region',
                         'road_conditions', 'participants_count', 'role', 'gender',
                         'violations', 'years_of_driving_experience', 'binary']].phik_matrix()
print('Корреляционная матрица с коэффициентом phi_k для переменной binary')
data_heatmap = correlation_matrix.loc[correlation_matrix.index != 'binary'][['binary']].sort_values(by='binary', ascending=False)

data_heatmap.style.background_gradient(cmap='coolwarm')

- violations наиболее высокая корреляция 0.72 - какие правила дорожного движения были нарушены
- role - 0.55 роль участника ДТП
- gender - 0.51 пол участника ДТП

In [ ]:
df.groupby('violations')['binary'].sum().sort_values(ascending=False).head()

Сильная корреляция со столбцом violations, по видимому обусловлена тем, что для ДТП с пострадавшими конкретное правило, которое было нарушено не указано

**Для оценки различия ДТП с пострадавшими и без будем использовать показатель доли пострадавших в зависимости от признака**

In [ ]:
df.groupby('role')['binary'].agg(['mean', 'count']).sort_values(by = 'mean', ascending=False).style.background_gradient(cmap='coolwarm')

In [ ]:
df.groupby('gender')['binary'].agg(['mean', 'count']).sort_values(by = 'mean', ascending=False).style.background_gradient(cmap='coolwarm')

In [ ]:
df.groupby('region')['binary'].agg(['mean', 'count']).sort_values(by = 'mean', ascending=False).head(10).style.background_gradient(cmap='coolwarm')

Отличие ДТП с пострадавшими коррелирует с такими признаками как нарушенное правило, роль, пол и регион
Наибольшая доля пострадавших в ДТП характерна для
- участников: велосипедист (0.99), пешеход (0.99), пассажир (0.94), для водителей не характерна (0.3)
- женщин (0.82) в существенно большей мере , чем для мужчин (0.46)
- регионов: вятско-полянский район (1.0), киропо-чепецкий район (0.87), шабалинский район (0.74)
- топе правил нарушенных при ДТП: 'переход через проезжую часть вне пешеходного перехода', 'несоответствие скорости конкретным условиям движения' 

### Общий вывод

- Наиболее типичное число участников ДТП 2.
- 59,3 % случаев ДТП происходит с 2 участниками, это вполне логично, учитывая, что как правило в ДТП учавствуют 2 стороны: водитель-водитель или водитель-пешеход, 20,2 % с 3 участниками, по-видимому с участием еще и пассажира, 10,6 с одним в случаях ДТП с одним ТС, например при сьезде с дороги.
- В 79,6 % случаев ДТП имеется один пострадавший, двое в 13,5 % происшествий, трое в 4,2 %
- Чаще всего на число пострадавших в ДТП влияет отсутствие, плохая различимость горизонтальной разметки проезжей части
- Самые частые категории ДТП `столкновение`, `наезд на пешехода` и `съезд с дороги`
- Самые опасные погодные условия `пасмурно`, `ясно`, `снегопад`, `дождь`
- Для мужчин и женщин в целом характерны приблизительно равные доли участия в ДТП.
- Участия женщин больше, чем мужчин в светлое время суток (64.4% и 63.8% соответственно) и темное с включенным освещением (22.4% и 21.8% соответственно)
- Участия мужчин больше чем женщин в сумерках (3.1% и 2.7% соответственно), в темное время суток, освещение не включено (1.5% и 1.3% соответственно) и в в темное время суток, освещение отсутствует (9.8% и 9.2% соответственно)
- В затемненных условиях участие в ДТП мужчин больше, а в условиях с освещением больше участие женщин

Отличие ДТП с пострадавшими коррелирует с такими признаками как нарушенное правило, роль, пол и регион
Наибольшая доля пострадавших в ДТП характерна для
- участников: велосипедист (0.99), пешеход (0.99), пассажир (0.94), для водителей не характерна (0.3)
- водителей женщин (0.82) в существенно большей мере , чем для мужчин (0.46)
- регионов: Вятско-полянский район (1.0), Киропо-чепецкий район (0.87), Шабалинский район (0.74)
- топе правил нарушенных при ДТП: 'переход через проезжую часть вне пешеходного перехода', 'несоответствие скорости конкретным условиям движения' 

**Рекомендации**
- Повысить уровень знания ПДД и безопасности для женщин
- Уделить внимание таким факторам как велосипедные дорожки, дорожная разметка и освещение дорог
- Особенно высокая аварийность с пострадавшими по Вятско-полянскому, Киропо-чепецкому и Шабалинскому районам